# Phase 7 — Construction d’un indicateur de risque climatique physique

## Contexte
Cette phase vise à mesurer l’exposition aux risques climatiques physiques
indépendamment du risque de crédit, à partir des données officielles FEMA
(National Risk Index).

## Objectif

- Préparer l’analyse d’impact sur le portefeuille de crédit (Phase 8)

## Livrable
- reports/tableau_exports/07_climate_risk_by_state.csv


In [1]:
import pandas as pd
import numpy as np


In [4]:
hazard_info = pd.read_csv("../data/raw/NRI_HazardInfo.csv", low_memory=False)
states = pd.read_csv("../data/raw/fema_nri_state.csv", low_memory=False)

hazard_info.shape, states.shape


((18, 13), (56, 313))

In [5]:
states.head()

,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,POPULATION,BUILDVALUE,AGRIVALUE,AREA,EAL_SCORE,...,WNTW_EALPE,WNTW_EALA,WNTW_EALT,WNTW_EALS,WNTW_EALR,WNTW_ALRB,WNTW_ALRP,WNTW_ALRA,WNTW_ALR_NPCTL,NRI_VER
0,1,S01,Alabama,AL,1,5014878,9.312787e+11,6.859652e+09,52949.28,75.000000,...,2.364710e+06,10407.071136,2.668945e+06,41.071429,Relatively Moderate,3.155108e-07,4.064990e-08,1.517143e-06,32.142857,March 2023
1,2,S02,Alaska,AK,2,731719,1.790662e+11,8.066034e+07,672485.30,25.000000,...,9.456492e+05,188.961662,9.623817e+05,28.571429,Relatively Low,9.238746e-08,1.114109e-07,2.342684e-06,66.071429,March 2023
2,3,S04,Arizona,AZ,4,7144933,1.210747e+12,4.418228e+09,115135.74,37.500000,...,8.926024e+05,0.002103,9.057077e+05,26.785714,Relatively Low,1.082416e-08,1.076966e-08,4.760000e-13,17.857143,March 2023
3,4,S05,Arkansas,AR,5,3007951,5.316416e+11,1.106954e+10,53735.26,58.928571,...,2.113970e+06,582034.811526,1.790300e+07,82.142857,Very High,2.860385e-05,6.058569e-08,5.257984e-05,91.071429,March 2023
4,5,S06,California,CA,6,39501350,6.898357e+12,5.179140e+10,165346.48,100.000000,...,9.937435e+05,0.016323,1.040743e+06,30.357143,Relatively Low,6.813204e-09,2.168724e-09,3.150000e-13,14.285714,March 2023


In [6]:
climate = states[[
    "STATE",
    "STATEABBRV",
    "EAL_SCORE"
]].copy()

climate.head()


,STATE,STATEABBRV,EAL_SCORE
0,Alabama,AL,75.000000
1,Alaska,AK,25.000000
2,Arizona,AZ,37.500000
3,Arkansas,AR,58.928571
4,California,CA,100.000000


In [7]:
climate["EAL_SCORE"].describe()


count     56.000000
mean      50.892857
std       29.124119
min        1.785714
25%       26.339286
50%       50.892857
75%       75.446429
max      100.000000
Name: EAL_SCORE, dtype: float64

In [8]:
#transformation des scores en catégories lisibles.

climate["climate_risk_level"] = pd.qcut(
    climate["EAL_SCORE"],
    q=5,
    labels=["Very Low", "Low", "Medium", "High", "Very High"]
)

climate["climate_risk_level"].value_counts()


climate_risk_level
Very Low     12
Low          11
Medium       11
High         11
Very High    11
Name: count, dtype: int64

In [9]:
final_climate = climate[
    ["STATEABBRV", "EAL_SCORE", "climate_risk_level"]
]

final_climate.head()


,STATEABBRV,EAL_SCORE,climate_risk_level
0,AL,75.000000,High
1,AK,25.000000,Low
2,AZ,37.500000,Low
3,AR,58.928571,Medium
4,CA,100.000000,Very High


In [10]:
final_climate.to_csv(
    "../reports/tableau_exports/07_climate_risk_by_state.csv",
    index=False
)

print("Export OK: reports/tableau_exports/07_climate_risk_by_state.csv")


Export OK: reports/tableau_exports/07_climate_risk_by_state.csv


## Interprétation

L’indicateur EAL_SCORE (Expected Annual Loss) fourni par FEMA permet de mesurer
l’exposition globale des États américains aux risques climatiques physiques.
La catégorisation en niveaux de risque facilite l’analyse territoriale et la
comparaison entre zones géographiques.
Cet indicateur constitue une brique exogène de risque climatique, utilisée
dans la phase suivante pour analyser l’impact du climat sur le risque de crédit.
